# Elastic Net

In this script, we will:
1. Load and preprocess the dataset.
2. Fit Elastic Net model.
3. Visualize top features per emotion class.
4. Identify features with zero coefficient value. 

In [45]:
# Import required libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.impute import SimpleImputer

### Task 1: Load the CSV file
Same as in script of correlation matrix

In [46]:
# Load CSV
df = pd.read_csv("../ravdess_processed_data/combined_summary_updated.csv") 
# Preview the dataset (first few rows)
display(df.head())

,f0_mean,f0_stddev,f0_range,f1_mean,f1_stddev,f1_range,f2_mean,f2_stddev,f2_range,f3_mean,...,AU23_std,AU24_std,AU25_std,AU26_std,AU28_std,AU43_std,mouth_openness_std,condition,id,sex
0,187.954488,71.416690,237.048375,922.823132,468.613940,1996.040116,1901.817187,625.338642,3267.843008,3056.335186,...,0.167865,0.027854,0.012677,0.105102,0.023936,0.137085,14.985116,happy,19,male
1,125.649480,18.592790,76.282777,896.902975,451.287522,2158.738997,1955.646204,547.505858,2528.638337,2989.774860,...,0.117064,0.181223,0.440511,0.322437,0.173739,0.039355,6.956635,calm,15,male
2,196.733026,59.763476,234.624127,807.336950,404.598851,1968.301265,1875.814368,424.990421,2902.265406,2966.456851,...,0.075628,0.028787,0.025267,0.234712,0.030841,0.009683,9.751621,surprised,21,male
3,233.731870,24.199943,101.728680,917.300413,424.694396,2083.518310,1890.668478,357.316926,2384.954571,2916.709624,...,0.182716,0.018268,0.002271,0.071390,0.032285,0.164527,12.347607,happy,9,male
4,263.508762,92.190807,402.873141,916.405078,487.460380,1964.002644,1998.063510,561.750355,2774.517885,3087.675790,...,0.109795,0.119696,0.181175,0.200143,0.069560,0.369833,12.062537,disgust,19,male


### Task 2: Create a function to preprocess the data

Follow the steps:
1. Drop metadata columns
2. Separate features and target
3. Encode target labels (sklearn - LabelEncoder)
4. Split into train and test sets (sklearn - train_test_split)
5. Impute missing values (sklearn - SimpleImputer)
6. Scale features (sklearn - StandardScaler)

In [47]:
# TODO: Implement a function for preprocessing
def preprocess_data(df, target_col="condition",test_size=0.2, random_state=42, metadata_cols=["id", "sex"]):
    """
        Preprocess the dataset for modeling:
        - Drop metadata columns
        - Separate features and target
        - Encode target labels
        - Split into train and test sets
        - Impute missing values (fit on train, transform on test)
        - Scale features (fit on train, transform on test)

        Args:
            df (pd.DataFrame): Input dataset
            target_col (str): Name of the target column
            metadata_cols (list, optional): Columns to drop
            test_size (float, optional): Proportion of the dataset to include in the test split
            random_state (int, optional): Seed used by the random number generator

        Returns:
            X_train_scaled (np.ndarray): Scaled training feature matrix
            X_test_scaled (np.ndarray): Scaled test feature matrix
            y_train (np.ndarray): Encoded training target labels
            y_test (np.ndarray): Encoded test target labels
            feature_names (list): Names of the features
            class_labels (np.ndarray): Original class labels
    """
    return None, None, None, None, None, None


<details>
<summary><span style="font-size:20px; color:darkgoldenrod; font-weight:bold;">Click to see the solution</span></summary>

```python
def preprocess_data(df, target_col="condition", metadata_cols=["id", "sex"]):
    """
    Preprocess the dataset for modeling - drop metadata columns, separate features and target, encode target labels, impute missing values, and scale features

    Args:
        df (pd.DataFrame): Input dataset
        target_col (str): Name of the target column
        metadata_cols (list, optional): Columns to drop. Defaults to ["id"].

    Returns:
        X_scaled (np.ndarray): Scaled feature matrix
        y_enc (np.ndarray): Encoded target labels
        feature_names (list): Names of the features
        class_labels (np.ndarray): Original class labels
    """
    if metadata_cols is None:
        metadata_cols = []

    # Drop metadata columns 
    df = df.drop(columns=metadata_cols, errors="ignore")

    # Separate target and features
    y = df[target_col]
    X = df.drop(columns=[target_col])
    feature_names = X.columns

    # Encode target labels
    label_enc = LabelEncoder()
    y_enc = label_enc.fit_transform(y)
    class_labels = label_enc.classes_

    # Split into train and test (before scaling/imputation to avoid leakage)
    X_train, X_test, y_train, y_test = train_test_split(
        X, y_enc, test_size=test_size, random_state=random_state, stratify=y_enc
    )

    # Impute missing values (fit on training, transform both)
    imputer = SimpleImputer(strategy="mean")
    X_train_imputed = imputer.fit_transform(X_train)
    X_test_imputed = imputer.transform(X_test)

    # Scale features (fit on training, transform both)
    scaler = StandardScaler()
    X_train_scaled = scaler.fit_transform(X_train_imputed)
    X_test_scaled = scaler.transform(X_test_imputed)

    return X_train_scaled, X_test_scaled, y_train, y_test, feature_names, class_labels

### Task 3: Elastic Net Logistic Regression

🎯 Fit multinomial logistic regression with Elastic Net penalty (sklearn - LogosticRegression)

Logistic Regression is suitable for classification tasks, including multinomial problems like predicting multiple categories

Elastic Net combines L1 (Lasso) and L2 (Ridge) regularization: [original paper: https://academic.oup.com/jrsssb/article/67/2/301/7109482]
1. L1 encourages sparsity, helping to select important features and ignore irrelevant ones.
2. L2 shrinks coefficients to prevent overfitting and improve model generalization.

⚠️ Important: Make sure to split the data into train and test sets, and use only the training set to identify feature importance—otherwise, you risk data leakage for the next classification task.

In [48]:
# TODO: Implement a function to for elastic net logisitic regression
def elasticnet_logreg(X_scaled, y_enc, feature_names, class_labels, l1_ratio=0.5, max_iter=5000):
    """
    Fit multinomial logistic regression with Elastic Net and return coefficients

    Args:
        X_scaled (np.ndarray): Preprocessed features 
        y_enc (np.ndarray): Encoded target labels
        feature_names (list): Names of the features
        class_labels (np.ndarray): Original class labels
        l1_ratio (float): Elastic Net mixing parameter
        max_iter (int): Maximum iterations for convergence

    Returns:
        coef_df (pd.DataFrame): Feature coefficients (features x classes)
    """
    return None
   

<details>
<summary><span style="font-size:20px; color:darkgoldenrod; font-weight:bold;">Click to see the solution</span></summary>

```python

def elasticnet_logreg(X_scaled, y_enc, feature_names, class_labels, l1_ratio=0.5, max_iter=5000):
    """
    Fit multinomial logistic regression with Elastic Net and return coefficients

    Args:
        X_scaled (np.ndarray): Preprocessed features 
        y_enc (np.ndarray): Encoded target labels
        feature_names (list): Names of the features
        class_labels (np.ndarray): Original class labels
        l1_ratio (float): Elastic Net mixing parameter
        max_iter (int): Maximum iterations for convergence

    Returns:
        coef_df (pd.DataFrame): Feature coefficients (features x classes)
    """
    clf = LogisticRegression(
        penalty="elasticnet",
        solver="saga",
        l1_ratio=l1_ratio,
        max_iter=max_iter,
        random_state=42,
        multi_class="multinomial"
    )
    clf.fit(X_scaled, y_enc)

    coef_df = pd.DataFrame(clf.coef_.T, index=feature_names, columns=class_labels)
    return coef_df
    

### Task 4: Plot Top Features

Visualize the top N features (by absolute coefficient) per emotion class using a heatmap.

In [49]:
def plot_coef_heatmap(coef_df, N=20, figsize=(10, 8), cmap="coolwarm"):
    """
    Plot a heatmap of the top N features (by absolute coefficient across classes).

    Args:
        coef_df (pd.DataFrame): Feature coefficients (features × classes)
        dataset_name (str): Dataset name for title
        N (int): Number of top features to display
        figsize (tuple): Figure size
        cmap (str): Colormap for heatmap

    Returns:
        None
    """
    try:
        # Select top-N features overall
        top_features = coef_df.abs().max(axis=1).nlargest(N).index
        plot_df = coef_df.loc[top_features]

        plt.figure(figsize=figsize)
        sns.heatmap(
            plot_df,
            annot=True, fmt=".2f", cmap=cmap, center=0,
            cbar_kws={"label": "Coefficient"}
        )
        plt.title(f"Top {N} Features (Elastic Net Logistic Regression)", fontsize=14)
        plt.xlabel("Emotion Class")
        plt.ylabel("Feature")
        plt.tight_layout()
        plt.show()
    except Exception as e:
        print(f"[Warning] Could not plot")
        print("Most likely the code is not yet complete (complete the cells with TODO).\n")
    


### Task 5: Apply Model 

Fit model and plot top features 

In [50]:
X_train_scaled, X_test_scaled, y_train, y_test, feature_names, class_labels = preprocess_data(df)
coefs = elasticnet_logreg(X_train_scaled, y_train, feature_names, class_labels, l1_ratio=0.5)

plot_coef_heatmap(coefs, N=15)

[Warning] Could not plot
Most likely the code is not yet complete (complete the cells with TODO).



### Insights?

🔍 Which features have the largest absolute coefficients for each emotion? Do these make sense?

### Task 6: Identify features with zero coefficient value. 

Relevant for the next script

In [51]:
try: 
    elasticnet_abs = coefs.abs().max(axis=1)
    elasticnet_dropped_features = elasticnet_abs[elasticnet_abs == 0].index.tolist()

    print("features dropped :", len(elasticnet_dropped_features), elasticnet_dropped_features)
except Exception as e:
    print("Most likely the code is not yet complete (complete the cells with TODO).\n")


Most likely the code is not yet complete (complete the cells with TODO).



#### Bonus task
🤔 Try experimenting with different l1_ratio values to see how it affects feature selection!